In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict
import numpy as np
from datetime import datetime, timedelta

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 5
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年03月10日"  # include
END_DATE = "2023年03月22日"  # include

WEEKS = [(START_DATE, END_DATE)]
current_date = START_DATE
while current_date < END_DATE:
    current_date_obj = datetime.strptime(current_date, '%Y年%m月%d日')
    next_date_obj = current_date_obj + timedelta(days=6)
    next_date = next_date_obj.strftime('%Y年%m月%d日')
    if next_date > END_DATE:
        next_date = END_DATE
    WEEKS.append((current_date, next_date))

    current_date_obj = next_date_obj + timedelta(days=1)
    current_date = current_date_obj.strftime('%Y年%m月%d日')
print(WEEKS)

TARGET_CATEGORY = "放逐_騎拉帝納VSTAR"

"""
放逐_騎拉帝納VSTAR

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR
LTB_快龍V

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_白馬VMAX
阿爾_鋁鋼龍VMAX
純阿爾VSTAR

夢幻VMAX

密勒頓ex
密勒頓ex_電柱
密勒頓ex_電柱_茸茸羊
密勒頓ex_茸茸羊

洛奇亞VSTAR
一擊_洛奇亞

沙奈朵ex
沙奈朵_蒼響
沙奈朵_超夢V-UNION

起源帕路奇亞VSTAR

連擊武道熊師VMAX

洗翠索羅亞克VSTAR

達克萊伊VSTAR

--

飄香豚ex

雷吉鐸拉戈VSTAR

烈空坐VMAX

火焰雞VMAX

密勒頓ex_other


起源帝牙盧卡VSTAR	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1
"""

[('2023年03月10日', '2023年03月22日'), ('2023年03月10日', '2023年03月16日'), ('2023年03月17日', '2023年03月22日')]


'\n放逐_騎拉帝納VSTAR\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\nLTB_快龍V\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_白馬VMAX\n阿爾_鋁鋼龍VMAX\n純阿爾VSTAR\n\n夢幻VMAX\n\n密勒頓ex\n密勒頓ex_電柱\n密勒頓ex_電柱_茸茸羊\n密勒頓ex_茸茸羊\n\n洛奇亞VSTAR\n一擊_洛奇亞\n\n沙奈朵ex\n沙奈朵_蒼響\n沙奈朵_超夢V-UNION\n\n起源帕路奇亞VSTAR\n\n連擊武道熊師VMAX\n\n洗翠索羅亞克VSTAR\n\n達克萊伊VSTAR\n\n--\n\n飄香豚ex\n\n雷吉鐸拉戈VSTAR\n\n烈空坐VMAX\n\n火焰雞VMAX\n\n密勒頓ex_other\n\n\n起源帝牙盧卡VSTAR\t1\n\n索羅風速狗\t1\n\n洛托姆VSTAR\t1\n\n月石\t1\n\n化石翼龍VSTAR\t1\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['夢幻VMAX', '交錯切換機', '化石翼龍VSTAR', 'LTB', 'LTB_勾魂眼_噴火龍', '雷吉艾勒奇VMAX', '密勒頓ex', '密勒頓ex_電柱', 'Other_Lost', 'LTB_洗翠黏美龍VSTAR', '達克萊伊VSTAR', '起源帕路奇亞VSTAR', '白馬蕾冠王VMAX', '放逐_騎拉帝納VSTAR', '超夢 V-UNION', '阿爾宙斯VSTAR', '光輝無極汰那', '飛翔皮卡丘VMAX', '酋雷姆VMAX', '起源帝牙盧卡VSTAR', 'LTB_空之封印石_水超鋼雷', 'LTB_空之封印石_水超闘雷', 'LTB_蓋歐卡', 'LTB_空之封印石_other', 'LTB_空之封印石_超鋼雷', '阿羅拉六尾VSTAR', '阿爾_白馬VMAX', '密勒頓ex_other', 'LTB_空之封印石_水超', '阿爾_飛天皮VMAX', '連擊武道熊師VMAX', 'LTB_空之封印石_超闘', 'LTB_空之封印石_水超鋼闘', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超雷', 'LTB_快龍V', 'LTB_空之封印石_悪水超鋼', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_草超鋼雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_超闘雷', 'LTB_空之封印石_草超鋼闘雷', 'LTB_空之封印石_超鋼闘雷', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超鋼', 'LTB_空之封印石_草超闘雷', 'LTB_空之封印石_水草超鋼雷', 'LTB_空之封印石_水草超鋼闘雷', 'LTB_空之封印石_悪水超闘', 'LTB_空之封印石_悪水超', '沙奈朵ex', 'LTB_空之封印石_超', '沙奈朵_蒼響', 'LTB_空之封印石_悪水超鋼闘', 'LTB_空之封印石_水草超雷', 'LTB_空之封印石_水炎超闘', 'LTB_空之封印石_悪水超闘雷', '密勒頓ex_電柱_茸茸羊', '密勒頓ex_茸茸羊', '洛托姆VSTAR', 'カラマネロVMAX', 'LTB_古月鳥', 'LTB_噴火龍', '純阿爾VSTAR', '阿爾_騎拉

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


6.604194641113281e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_target_deck = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_target_deck = deck_cnt

In [11]:
deck_cnt_list = sorted(deck_cnt_list, reverse=True)
for num_deck, deck_name in deck_cnt_list:
    if num_deck > 0:
        print(f"{deck_name}\t{num_deck}\n")

LTB	191

放逐_騎拉帝納VSTAR	144

洛奇亞VSTAR	129

阿爾宙斯VSTAR	119

一擊_洛奇亞	113

夢幻VMAX	108

密勒頓ex	104

雷吉艾勒奇VMAX	94

沙奈朵ex	89

飛翔皮卡丘VMAX	88

沙奈朵_蒼響	87

交錯切換機	75

LTB_勾魂眼_噴火龍	62

密勒頓ex_電柱	50

Other_騎拉帝納VSTAR	49

密勒頓ex_電柱_茸茸羊	44

LTB_空之封印石_水超闘雷	38

阿爾_飛天皮VMAX	36

阿爾_鋁鋼龍VMAX	32

超夢 V-UNION	30

LTB_蓋歐卡	30

連擊武道熊師VMAX	29

沙奈朵_超夢V-UNION	28

阿爾_騎拉帝納VSTAR	20

LTB_快龍V	20

洗翠索羅亞克VSTAR	15

others	15

白馬蕾冠王VMAX	13

LTB_空之封印石_水超雷	11

LTB_洗翠黏美龍VSTAR	11

黑馬蕾冠王VMAX	10

起源帕路奇亞VSTAR	10

LTB_空之封印石_other	10

起源帝牙盧卡VSTAR	8

純阿爾VSTAR	8

柱神	8

密勒頓ex_茸茸羊	8

LTB_空之封印石_水超鋼雷	8

酋雷姆VMAX	6

Other_Lost	6

光輝無極汰那	5

雷吉鐸拉戈VSTAR	4

阿爾_白馬VMAX	4

達克萊伊VSTAR	4

化石翼龍VSTAR	4

LTB_空之封印石_水超闘	4

LTB_空之封印石_水超鋼闘	4

洛托姆VSTAR	3

幻影索羅亞克	2

密勒頓ex_other	2

LTB_空之封印石_水超鋼闘雷	2

阿爾_雷吉鐸拉戈VSTAR	1

烈空坐VMAX	1

火焰雞VMAX	1

月石	1

ヒスイ ダイケンキVSTAR	1

LTB_空之封印石_超鋼闘雷	1

LTB_空之封印石_水超鋼	1

LTB_空之封印石_水草超雷	1

LTB_空之封印石_水炎超闘	1

LTB_空之封印石_悪水超闘雷	1

LTB_空之封印石_悪水超	1



In [12]:
# number of decks for the target category
print(number_target_deck)
if number_target_deck <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

144


In [13]:
def fill_na(col):
    if col.name not in ["date", "prefecture"]:
        return col.fillna(0)
    else:
        return col

In [14]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]
# weeks = [
#     ("2023年02月04日", "2023年03月10日"),  # total
#     ("2023年02月04日", "2023年02月10日"),  # week1 ...
#     ("2023年02月11日", "2023年02月17日"),
#     ("2023年02月18日", "2023年02月24日"),
#     ("2023年02月25日", "2023年03月03日"),
#     ("2023年03月04日", "2023年03月10日"),
# ]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str < START_DATE or date_exclude_jp_str > END_DATE:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.apply(fill_na)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # Calculate for each week
    for week_id, week in enumerate(WEEKS):
        week_name = f"week_{week_id}" if week_id > 0 else "all"
        week_start, week_end = week
        df_week = df.loc[(df['date'] >= week_start) & (df['date'] <= week_end)]

        # calculate
        num_decks = df_week.shape[0]
        num_used = df_week[col_list].sum(axis="rows", numeric_only=True)
        num_picked = df_week[col_list].astype(bool).sum(axis="rows")
        avg_num_used = num_used / num_picked
        pick_rate = num_picked / num_decks

        # insert rows in df
        avg_idx = f"avg_num_used_{week_name}\n{week[0]}\n{week[1]}"
        pick_idx = f"pick_rate_{week_name}\n{week[0]}\n{week[1]}"
        df.loc[avg_idx] = {}
        df.loc[pick_idx] = {}
        for col in col_list:
            df.loc[avg_idx, col] = avg_num_used[col] if not np.isnan(avg_num_used[col]) else 0
            df.loc[pick_idx, col] = pick_rate[col] if not np.isnan(pick_rate[col]) else 0

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    num_weeks = len(WEEKS)
    # target_rows = [num_rows - 1, num_rows - 2]
    target_rows = [num_rows - i for i in range(num_weeks*2-1, 0, -2)]
    target_rows += [num_rows - i for i in range(num_weeks*2, 1, -2)]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values(f"pick_rate_all\n{WEEKS[0][0]}\n{WEEKS[0][1]}", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [15]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{START_DATE}-{END_DATE}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.3f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if "avg_num_used" in deck_id or "pick_rate" in deck_id:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

/Users/blake.pan/anaconda3/envs/sandbox_crawler/lib/python3.7/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [16]:
# df_list[0]

In [17]:
# df_list[1]

In [18]:
# df_list[2]

In [19]:
# df_list[3]

In [20]:
# df_list[4]

In [21]:
other_decks = decks["others"]
other_decks = sorted(other_decks, key=lambda deck: deck["date"], reverse=True)
# print(other_decks)
for d in other_decks:
    date = d["date"]
    link = d["deck_link"]
    print(f"{date}\t{link}")

2023年03月19日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/XpMpyS-HnyFKv-UMX3yy
2023年03月18日(土)	https://www.pokemon-card.com/deck/confirm.html/deckID/gNNPnn-2iyHFM-nLnnHH
2023年03月18日(土)	https://www.pokemon-card.com/deck/confirm.html/deckID/VFfFfF-fX93Vd-vkdkk1
2023年03月17日(金)	https://www.pokemon-card.com/deck/confirm.html/deckID/FFkvfd-doOe6R-dVFvfk
2023年03月16日(木)	https://www.pokemon-card.com/deck/confirm.html/deckID/HLggnL-2zX61Y-iHNQN9
2023年03月12日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/gQLnNn-ChoYFB-9HNPLg
2023年03月12日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/fFffkF-E38V2z-fVkVVk
2023年03月12日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/MM3EXy-ggdxp6-p2pXy3
2023年03月12日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/kfFVFk-Au38nP-fkfVwV
2023年03月12日(日)	https://www.pokemon-card.com/deck/confirm.html/deckID/xJxDY8-U8kyNi-DDcYc8
2023年03月11日(土)	https://www.pokemon-card.com/deck/confirm.html/deckID/8DcK8c-v3K6je-DcDccx
2023年03月11